- These inference notebook is based on @nischaydnk's [notebook](https://www.kaggle.com/code/nischaydnk/bengali-finetuning-baseline-wav2vec2-inference). If it's helpful to you, please upvote his firstly.
- Running this notebook you can score 0.338 on leaderboard.
- This model is trained around 8 epochs with 1e-4 learning rate on the data of [Common Voice 13](http://www.kaggle.com/datasets/umongsain/common-voice-13-bengali-normalized) provided by @UMONG SAIN and the competition data 'valid' part.

In [1]:
!cp -r ../input/python-packages2 ./

!tar xvfz ./python-packages2/jiwer.tgz
!pip install ./jiwer/jiwer-2.3.0-py3-none-any.whl -f ./ --no-index
!tar xvfz ./python-packages2/normalizer.tgz
!pip install ./normalizer/bnunicodenormalizer-0.0.24.tar.gz -f ./ --no-index
!tar xvfz ./python-packages2/pyctcdecode.tgz
!pip install ./pyctcdecode/attrs-22.1.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/exceptiongroup-1.0.0rc9-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/hypothesis-6.54.4-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pygtrie-2.5.0.tar.gz -f ./ --no-index --no-deps
!pip install ./pyctcdecode/sortedcontainers-2.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pyctcdecode-0.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps

!tar xvfz ./python-packages2/pypikenlm.tgz
!pip install ./pypikenlm/pypi-kenlm-0.1.20220713.tar.gz -f ./ --no-index --no-deps

jiwer/
jiwer/jiwer-2.3.0-py3-none-any.whl
jiwer/python-Levenshtein-0.12.2.tar.gz
jiwer/setuptools-65.3.0-py3-none-any.whl
Looking in links: ./
Processing ./jiwer/jiwer-2.3.0-py3-none-any.whl
INFO: pip is looking at multiple versions of jiwer to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement python-Levenshtein==0.12.2 (from jiwer) (from versions: none)
ERROR: No matching distribution found for python-Levenshtein==0.12.2
normalizer/
normalizer/bnunicodenormalizer-0.0.24.tar.gz
Looking in links: ./
Processing ./normalizer/bnunicodenormalizer-0.0.24.tar.gz
  Preparing metadata (setup.py) ... - done
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.0.24-py3-none-any.whl size=17628 sha256=6531327b41d150598935f6d9d0d076b190c6d8e1b48cd791afafe4ecc3eebe44
  Stored in directory: /root/.cache/pip/wheels/78/d7/75/6986dc3616718f950b80e3bd79a796ef618eaef6cd800e7909
Successf

In [2]:
rm -r python-packages2 jiwer normalizer pyctcdecode pypikenlm

In [3]:
import typing as tp
from pathlib import Path
from functools import partial
from dataclasses import dataclass, field

import pandas as pd
import pyctcdecode
import numpy as np
from tqdm.notebook import tqdm

import librosa

import pyctcdecode
import kenlm
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC
from bnunicodenormalizer import Normalizer

import cloudpickle as cpkl

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
ROOT = Path.cwd().parent
INPUT = ROOT / "input"
DATA = INPUT / "bengaliai-speech"
TRAIN = DATA / "train_mp3s"
TEST = DATA / "test_mp3s"

SAMPLING_RATE = 16_000
MODEL_PATH = INPUT / "bengali-ai-wave-2-vec-3"
LM_PATH = INPUT / "arijitx-full-model/wav2vec2-xls-r-300m-bengali/language_model"

### load model, processor, decoder

In [5]:
model = Wav2Vec2ForCTC.from_pretrained(MODEL_PATH)
processor = Wav2Vec2Processor.from_pretrained(MODEL_PATH)

/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:53: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:51   │
│ in from_pretrained                                                                               │
│                                                                                                  │
│    48 │   @classmethod                                                                           │
│    49 │   def from_pretrained(cls, pretrained_model_name_or_path, **kwargs):                     │
│    50 │   │   try:                                                                               │
│ ❱  51 │   │   │   return super().from_pretrained(pretrained_model_name_or_path, **kwargs)        │
│    52 │   │   except OSError:                                                                    │
│    53 │   │   │   warnings.warn(                                                                 │
│    54 │   │   │   │   f"Loading a tokenizer inside {cls.__name__} from a config that does not"   │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/processing_utils.py:184 in from_pretrained  │
│                                                                                                  │
│   181 │   │   │   │   [`~feature_extraction_utils.FeatureExtractionMixin.from_pretrained`] and   │
│   182 │   │   │   │   [`~tokenization_utils_base.PreTrainedTokenizer.from_pretrained`].          │
│   183 │   │   """                                                                                │
│ ❱ 184 │   │   args = cls._get_arguments_from_pretrained(pretrained_model_name_or_path, **kwarg   │
│   185 │   │   return cls(*args)                                                                  │
│   186 │                                                                                          │
│   187 │   @classmethod                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/processing_utils.py:228 in                  │
│ _get_arguments_from_pretrained                                                                   │
│                                                                                                  │
│   225 │   │   │   else:                                                                          │
│   226 │   │   │   │   attribute_class = getattr(transformers_module, class_name)                 │
│   227 │   │   │                                                                                  │
│ ❱ 228 │   │   │   args.append(attribute_class.from_pretrained(pretrained_model_name_or_path, *   │
│   229 │   │   return args                                                                        │
│   230 │                                                                                          │
│   231 │   @property                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:712 in     │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   709 │   │   │   │   return tokenizer_class_fast.from_pretrained(pretrained_model_name_or_pat   │
│   710 │   │   │   else:                                                                          │
│   711 │   │   │   │   if tokenizer_class_py is not None:                                         │
│ ❱ 712 │   │   │   │   │   return tokenizer_class_py.from_pr

In [ ]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

decoder = pyctcdecode.build_ctcdecoder(
    list(sorted_vocab_dict.keys()),
    str(LM_PATH / "5gram.bin"),
)

In [ ]:
processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

## prepare dataloader

In [ ]:
class BengaliSRTestDataset(torch.utils.data.Dataset):
    
    def __init__(
        self,
        audio_paths: list[str],
        sampling_rate: int
    ):
        self.audio_paths = audio_paths
        self.sampling_rate = sampling_rate
        
    def __len__(self,):
        return len(self.audio_paths)
    
    def __getitem__(self, index: int):
        audio_path = self.audio_paths[index]
        sr = self.sampling_rate
        w = librosa.load(audio_path, sr=sr, mono=False)[0]
        
        return w

In [ ]:
test = pd.read_csv(DATA / "sample_submission.csv", dtype={"id": str})
print(test.head())

In [ ]:
test_audio_paths = [str(TEST / f"{aid}.mp3") for aid in test["id"].values]

In [ ]:
test_dataset = BengaliSRTestDataset(
    test_audio_paths, SAMPLING_RATE
)

collate_func = partial(
    processor_with_lm.feature_extractor,
    return_tensors="pt", sampling_rate=SAMPLING_RATE,
    padding=True,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=16, shuffle=False,
    num_workers=2, collate_fn=collate_func, drop_last=False,
    pin_memory=True,
)

## Inference

In [ ]:
if not torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device("cuda")
print(device)

In [ ]:
model = model.to(device)
model = model.eval()
model = model.half()

In [ ]:
pred_sentence_list = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        x = batch["input_values"]
        x = x.to(device, non_blocking=True)
        with torch.cuda.amp.autocast(True):
            y = model(x).logits
        y = y.detach().cpu().numpy()
        
        for l in y:  
            sentence = processor_with_lm.decode(l, beam_width=512).text
            pred_sentence_list.append(sentence)

## Make Submission

In [ ]:
bnorm = Normalizer()

def postprocess(sentence):
    period_set = set([".", "?", "!", "।"])
    _words = [bnorm(word)['normalized']  for word in sentence.split()]
    sentence = " ".join([word for word in _words if word is not None])
    try:
        if sentence[-1] not in period_set:
            sentence+="।"
    except:
        # print(sentence)
        sentence = "।"
    return sentence

In [ ]:
pp_pred_sentence_list = [postprocess(s) for s in tqdm(pred_sentence_list)]

In [ ]:
test["sentence"] = pp_pred_sentence_list

test.to_csv("submission.csv", index=False)

display(test.head())